In [ ]:
!pip3 install torch
!pip3 install pandas
!pip3 install numpy
!pip3 install matplotlib
!pip3 install scikit-learn
!pip3 install transformers
!pip3 install seaborn

In [1]:
import os, sys
sys.path.insert(1, '../dataset')
import numpy as np
import pandas as pd
import sklearn
import warnings

warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import BertTokenizer
import torch
from utils import tag_to_word_df
import torch
import argparse
import utils

test the ATE model with the testing data, this process may require some modifications later, this is to check if the trained model works for our dataset

In [ ]:

# def replace_sentiment_tags (tags):
#     return [-1 if tag == 'O' else 2 if tag.endswith('positive') else 0 if tag.endswith('negative') else 1 if tag.endswith('neutral') else tag for tag in tags]


In [ ]:

# parser = argparse.ArgumentParser()
# parser.add_argument('--batch', type=int, default=8, help='batch size')
# parser.add_argument('--epochs', type=int, default=5, help='number of epochs')
# parser.add_argument('--lr', type=float, default=3*1e-5, help='learning rate')
# parser.add_argument('--lr_schedule', type=bool, default=True, help='learning rate scheduler')
# parser.add_argument('--adapter', type=bool, default=False, help='adapter')
batch = 8
epochs = 5
lr = 3*1e-5
lr_schedule = True
adapter = False

def train_ABTE (batch, epochs, lr, lr_schedule, adapter):

    #load
    data = pd.read_csv('../dataset/df_label_539980_train_20240705.csv')

    data['bio_tags'] = data['bio_tags'].apply(utils.convert_to_array)
    data['sentiment_tags'] = data['sentiment_tags'].apply(utils.convert_to_array)
    data['tokens'] = data['tokens'].apply(utils.convert_to_array)

    data['bio_tags'] = data['bio_tags'].apply(utils.replace_SEP).apply(utils.replace_tags)
    data['sentiment_tags'] = data['sentiment_tags'].apply(utils.replace_SEP).apply(utils.replace_sentiment_tags)
    from transformers import BertTokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    
    # if torch.backends.mps.is_available():
    #     DEVICE = torch.device("mps")
    # else:
    #     DEVICE = torch.device("cpu")
    DEVICE = torch.device("cpu")
    from abte import ABTEModel
    modelABTE = ABTEModel(tokenizer, adapter)
    modelABTE.train(data, batch_size=batch, lr=lr, epochs=epochs, device=DEVICE, lr_schedule=lr_schedule)

train_ABTE(batch, epochs, lr, lr_schedule, adapter)
print('Done')

In [ ]:
from abte import ABTEModel
from utils import convert_to_array,replace_SEP,replace_tags, replace_sentiment_tags

batch = 8
lr = 3*1e-5
epochs = 5

def run_ABTE_test_train(adapter, lr_schedule):
    if adapter:
        if lr_schedule: dir_name  = "model_ABTE_adapter_scheduler"
        else: dir_name = "model_ABTE_adapter"
    else:
        if lr_schedule: dir_name  = "model_ABTE_scheduler"
        else: dir_name = "model_ABTE"
    #load
    data = pd.read_csv('../dataset/df_label_539980_train_20240705.csv')
    data_test = pd.read_csv('../dataset/df_label_539980_test_20240705.csv')

    data['bio_tags'] = data['bio_tags'].apply(convert_to_array)
    data['sentiment_tags'] = data['sentiment_tags'].apply(convert_to_array)
    data['tokens'] = data['tokens'].apply(convert_to_array)

    data['bio_tags'] = data['bio_tags'].apply(replace_SEP).apply(replace_tags)
    data['sentiment_tags'] = data['sentiment_tags'].apply(replace_SEP).apply(replace_sentiment_tags)

    data_test['bio_tags'] = data_test['bio_tags'].apply(convert_to_array)
    data_test['sentiment_tags'] = data_test['sentiment_tags'].apply(convert_to_array)
    data_test['tokens'] = data_test['tokens'].apply(convert_to_array)

    data_test['bio_tags'] = data_test['bio_tags'].apply(replace_SEP).apply(replace_tags)
    data_test['sentiment_tags'] = data_test['sentiment_tags'].apply(replace_SEP).apply(replace_sentiment_tags)

    # define parameters for model
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    
    # DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    DEVICE = torch.device("cpu")
    # define model
    modelABTE = ABTEModel(tokenizer, adapter=adapter)

    # load model and predict
    model_path = dir_name+'/model_lr3.0000000000000004e-05_epochs4_batch8.pkl'
    test_accuracy, test_report = modelABTE.test(data_test, load_model=model_path, device=DEVICE)
    test_pred, test_targets = modelABTE.predict_batch(data_test, load_model=model_path, device=DEVICE)

    train_accuracy, train_report = modelABTE.test(data, load_model=model_path, device=DEVICE)
    train_pred, train_targets = modelABTE.predict_batch(data, load_model=model_path, device=DEVICE)

    print('*****test_report')
    print(test_report)
    print('****train_report')
    print(train_report)


    #save results
    if not os.path.exists(dir_name+'/results'):
        os.makedirs(dir_name+'/results')
        print('folder not exists, created result folder')

    #report
    with open(dir_name+'/results/test_report_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), 'w') as f:
        for r in test_report.split('\n'):
            f.write(r + '\n')

    with open(dir_name+'/results/train_report_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), 'w') as f:
        for r in train_report.split('\n'):
            f.write(r + '\n')

    #predictions
    data_test['Predicted'] = test_pred
    data_test['Actual'] = test_targets
    data_test.to_csv(dir_name+'/results/test_pred_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), index=False)

    data['Predicted'] = train_pred
    data['Actual'] = train_targets
    data.to_csv(dir_name+'/results/train_pred_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), index=False)

    #accuracy
    test_accuracy = np.array(test_accuracy)
    train_accuracy = np.array(train_accuracy)

    with open(dir_name+'/results/test_accuracy_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), 'w') as f:
        f.write(str(test_accuracy))
    with open(dir_name+'/results/train_accuracy_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), 'w') as f:
        f.write(str(train_accuracy))
    
    print('completed!!!!! huurayyyyyy!!!!')

In [ ]:
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    
# DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# model_path = 'model_ABTE_scheduler/model_lr3.0000000000000004e-05_epochs4_batch8.pkl'

# # define model
# modelABTE = ABTEModel(tokenizer, False)
# word_pieces, predictions, outputs = modelABTE.predict('It was a little tighter in the chest area than i thought it would be, but i still like it very much!', load_model=model_path, device=DEVICE)

# print(word_pieces)
# print(predictions)

In [ ]:
run_ABTE_test_train(False, True)

In [ ]:
# *****test_report
#               precision    recall  f1-score   support

#         none       0.99      0.99      0.99      1883
#  start of AT       0.74      0.71      0.72        55
#   mark of AT       0.00      0.00      0.00         0

#    micro avg       0.98      0.98      0.98      1938
#    macro avg       0.58      0.57      0.57      1938
# weighted avg       0.98      0.98      0.98      1938

# ****train_report
#               precision    recall  f1-score   support

#         none       1.00      1.00      1.00      7564
#  start of AT       0.91      0.89      0.90       224
#   mark of AT       0.00      0.00      0.00         0

#    micro avg       0.99      0.99      0.99      7788
#    macro avg       0.64      0.63      0.63      7788
# weighted avg       0.99      0.99      0.99      7788


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    
    # DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
DEVICE = torch.device("cpu")
    # define model
    # load model and predict
model_path = 'model_ABTE_scheduler'+'/model_lr3.0000000000000004e-05_epochs4_batch8.pkl'
    
# DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# model_path = 'model_ABTE_scheduler/model_lr3.0000000000000004e-05_epochs4_batch8.pkl'

# define model
modelABTE = ABTEModel(tokenizer, False)
word_pieces, predictions, outputs = modelABTE.predict('It was a little tighter in the chest area than i thought it would be, but i still like it very much!', load_model=model_path, device=DEVICE)

print(word_pieces)
print(predictions)

In [ ]:
# modelABTE = ABTEModel(tokenizer, False)
# %time train_model_ATE(train_loader, 3)
# model_ATE = load_model(model_ATE, 'bert_ATE.pkl')
# %time x, y = test_model_ATE(test_loader)
# print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

In [5]:
batch = 8
epochs = 5
lr = 3*1e-5
lr_schedule = True
adapter = False

def train_ABSA (batch, epochs, lr, lr_schedule, adapter):

    #load
    data = pd.read_csv('../dataset/df_label_539980_train_20240705.csv')

    data['bio_tags'] = data['bio_tags'].apply(utils.convert_to_array)
    data['sentiment_tags'] = data['sentiment_tags'].apply(utils.convert_to_array)
    data['tokens'] = data['tokens'].apply(utils.convert_to_array)

    data['bio_tags'] = data['bio_tags'].apply(utils.replace_SEP).apply(utils.replace_tags)
    data['sentiment_tags'] = data['sentiment_tags'].apply(utils.replace_SEP).apply(utils.replace_sentiment_tags)

    from transformers import BertTokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    
    DEVICE = torch.device("cpu")
    
    from absa import ABSAModel
    modelABSA = ABSAModel(tokenizer, adapter=True)
    modelABSA.train(data, batch_size=batch, lr=lr, epochs=epochs, device=DEVICE, lr_schedule=True)

train_ABSA(batch, epochs, lr, lr_schedule, adapter)
print('Done')

Training model...
Learning rate scheduler:  True
Adapter:  True
epoch: 0	batch: 1/16	loss: 1.1772578954696655	batch time: 1.442	total time: 1.442
epoch: 0	batch: 2/16	loss: 1.107589840888977	batch time: 1.59	total time: 3.032
epoch: 0	batch: 3/16	loss: 1.1904220581054688	batch time: 1.251	total time: 4.2829999999999995
epoch: 0	batch: 4/16	loss: 1.185410976409912	batch time: 0.611	total time: 4.893999999999999
epoch: 0	batch: 5/16	loss: 1.1009628772735596	batch time: 0.635	total time: 5.528999999999999
epoch: 0	batch: 6/16	loss: 1.1719692945480347	batch time: 0.579	total time: 6.107999999999999
epoch: 0	batch: 7/16	loss: 1.2508803606033325	batch time: 0.661	total time: 6.768999999999998
epoch: 0	batch: 8/16	loss: 1.091991662979126	batch time: 0.617	total time: 7.385999999999998
epoch: 0	batch: 9/16	loss: 1.1399116516113281	batch time: 0.785	total time: 8.170999999999998
epoch: 0	batch: 10/16	loss: 1.2514110803604126	batch time: 1.397	total time: 9.567999999999998
epoch: 0	batch: 11/16	

In [6]:
from absa import ABSAModel
from utils import convert_to_array,replace_SEP,replace_tags, replace_sentiment_tags

#save results
batch = 8
lr = 1e-5
epochs = 5

def run_ABSA_test_train(adapter, lr_schedule):
    if adapter:
        if lr_schedule: dir_name_s  = "model_ABSA_adapter_scheduler"
        else: dir_name_s = "model_ABSA_adapter"
    else:
        if lr_schedule: dir_name_s  = "model_ABSA_scheduler"
        else: dir_name_s = "model_ABSA"
    print(dir_name_s)

    #load
    data = pd.read_csv('../dataset/df_label_539980_train_20240705.csv')
    data_test = pd.read_csv('../dataset/df_label_539980_test_20240705.csv')

    data['bio_tags'] = data['bio_tags'].apply(convert_to_array)
    data['sentiment_tags'] = data['sentiment_tags'].apply(convert_to_array)
    data['tokens'] = data['tokens'].apply(convert_to_array)

    data['bio_tags'] = data['bio_tags'].apply(replace_SEP).apply(replace_tags)
    data['sentiment_tags'] = data['sentiment_tags'].apply(replace_SEP).apply(replace_sentiment_tags)

    data_test['bio_tags'] = data_test['bio_tags'].apply(convert_to_array)
    data_test['sentiment_tags'] = data_test['sentiment_tags'].apply(convert_to_array)
    data_test['tokens'] = data_test['tokens'].apply(convert_to_array)

    data_test['bio_tags'] = data_test['bio_tags'].apply(replace_SEP).apply(replace_tags)
    data_test['sentiment_tags'] = data_test['sentiment_tags'].apply(replace_SEP).apply(replace_sentiment_tags)

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    DEVICE = torch.device("cpu")

    modelABSA = ABSAModel(tokenizer, adapter=adapter)

    model_path = dir_name_s+'/model_lr3.0000000000000004e-05_epochs4_batch8.pkl'
    test_accuracy, test_report = modelABSA.test(data_test, load_model=model_path, device=DEVICE)
    test_pred, test_pol = modelABSA.predict_batch(data_test, load_model=model_path, device=DEVICE)

    train_accuracy, train_report = modelABSA.test(data, load_model=model_path, device=DEVICE)
    train_pred, train_pol = modelABSA.predict_batch(data, load_model=model_path, device=DEVICE)

    print('*****test_report')
    print(test_report)
    print('****train_report')
    print(train_report)

    #save results
    if not os.path.exists(dir_name_s+'/results'):
        os.makedirs(dir_name_s+'/results')

    #report
    with open(dir_name_s+'/results/test_report_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), 'w') as f:
        for r in test_report.split('\n'):
            f.write(r + '\n')

    with open(dir_name_s+'/results/train_report_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), 'w') as f:
        for r in train_report.split('\n'):
            f.write(r + '\n')

    #predictions
    data_test['Predicted'] = test_pred
    data_test['Actual'] = test_pol
    data_test.to_csv(dir_name_s+'/results/test_pred_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), index=False)

    data['Predicted'] = train_pred
    data['Actual'] = train_pol
    data.to_csv(dir_name_s+'/results/train_pred_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), index=False)

    #accuracy
    test_accuracy = np.array(test_accuracy)
    train_accuracy = np.array(train_accuracy)

    with open(dir_name_s+'/results/test_accuracy_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), 'w') as f:
        f.write(str(test_accuracy))
    with open(dir_name_s+'/results/train_accuracy_lr{}_epochs{}_batch{}.csv'.format(lr, epochs, batch), 'w') as f:
        f.write(str(train_accuracy))


In [7]:
run_ABSA_test_train(True, True)

model_ABSA_adapter_scheduler


100%|██████████| 132/132 [00:29<00:00,  4.43it/s]

*****test_report
              precision    recall  f1-score   support

    negative       0.68      0.71      0.70        21
     neutral       0.00      0.00      0.00         1
    positive       0.50      0.50      0.50        12

    accuracy                           0.62        34
   macro avg       0.39      0.40      0.40        34
weighted avg       0.60      0.62      0.61        34

****train_report
              precision    recall  f1-score   support

    negative       0.93      1.00      0.96        88
     neutral       0.00      0.00      0.00         4
    positive       0.95      0.88      0.91        40

    accuracy                           0.93       132
   macro avg       0.62      0.62      0.62       132
weighted avg       0.90      0.93      0.92       132

